## 1. Load the dataset



To run this notebook, you will need to install: pandas, openai, transformers, plotly, matplotlib, scikit-learn, torch (transformer dep), torchvision, and scipy.

In [13]:
import pandas as pd
import openai

# Setup API-Key for OpenAi
openai.api_key = "sk-"

# Define the embedding model which is used
embedding_model = "text-embedding-ada-002"

# Retrieve the data from the database
input_datapath = pd.read_sql_query("SELECT * FROM 'mortgage complaints'", "sqlite:///StaterData.db")

# Limit test size due to performance issues
data = input_datapath.loc[:30]

# Extract the relevant columns and put into a list
texts = data["Consumer complaint narrative"].tolist()

# Initialize a list to store the embeddings
embeddings = []

# Generate embeddings for the texts
for text in texts:
    response = openai.Completion.create(
        engine=embedding_model,
        prompt=text,
        max_tokens=1,
        temperature=0,
        top_p=0,
        n=1,
        stop=None,
        logprobs=0,
    )
    embedding = response.choices[0].logprobs.token_logprobs[0]
    embeddings.append(embedding)

# Assign the embeddings to the DataFrame
data["Embeddings"] = embeddings

# data = data["Embeddings"]

# Save the DataFrame with embeddings to a CSV file
data.to_csv("MortgageEmbeddings.csv", index=False)



C:\Users\Lars\AppData\Local\Temp\ipykernel_13256\2459883123.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["Embeddings"] = embeddings


In [15]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
import numpy as np
from sklearn.model_selection import train_test_split

# Load the data with embeddings from the CSV file
data = pd.read_csv("MortgageEmbeddings.csv")

# Prepare the feature matrix X and target vector y
X = data["Embeddings"].tolist()
y = data["Issue"]  # Replace "TargetVariable" with the actual name of your target variable

X = np.array(X).reshape(-1, 1)

print(X)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create an instance of the random forest classifier
rf_classifier = RandomForestClassifier()

# Train the random forest classifier
rf_classifier.fit(X_train, y_train)

# Make predictions on the test data
y_pred = rf_classifier.predict(X_test)

# Evaluate the performance of the model
accuracy = (y_pred == y_test).mean()
print("Accuracy:", accuracy)


[[-2.5349712]
 [-1.7125219]
 [-1.941894 ]
 [-1.1566694]
 [-1.9006227]
 [-2.4283676]
 [-1.9624331]
 [-2.1549342]
 [-2.101875 ]
 [-1.6315789]
 [-1.7694852]
 [-1.6135703]
 [-1.7266814]
 [-2.2935843]
 [-1.9359815]
 [-2.6362653]
 [-1.5304936]
 [-1.734073 ]
 [-2.1274507]
 [-2.7574806]
 [-1.8576968]
 [-1.6701773]
 [-2.5826762]
 [-2.0674033]
 [-1.4728855]
 [-2.0358458]
 [-2.0172763]
 [-2.4396744]
 [-1.3477863]
 [-2.6027756]
 [-2.2203102]]
Accuracy: 0.14285714285714285
